In [ ]:
import pandas as pd
import base64
import re

In [ ]:
def decode_base64(word):
    try:
        return base64.b64decode(word).decode("utf-8")
    except:
        pass

In [ ]:
with open("features.txt", "r") as f:
    vocabulary = f.read().splitlines()

In [ ]:
def split_by_vocabulary(string):
    pattern = '|'.join(map(re.escape, vocabulary))
    matches = re.findall(pattern, string)
    return matches

In [ ]:
def decode_session(full_session):
    new_full_session = []
    for session_chunck in full_session.split(";"):
        if "base64 --decode" in session_chunck or "echo" in session_chunck:
            for word in session_chunck.split("\""):
                decode = decode_base64(word)
                if decode:
                    words_decoded = decode.split("\n")
                    # remove shebang
                    if words_decoded[0] == "#!/bin/bash":
                        words_decoded[0] = ""
                    new_full_session.append("; ".join(words_decoded))
                else:
                    new_full_session.append(session_chunck)
        else:
            new_full_session.append(session_chunck)
    return "; ".join(new_full_session)

In [ ]:
df_original = pd.read_parquet('ssh_attacks.parquet')
df_decoded=df_original.copy()
df_decoded['first_timestamp'] = pd.to_datetime(df_decoded['first_timestamp'])
df_decoded["full_session"] = df_decoded["full_session"].apply(lambda session: decode_session(session))
df_decoded.to_parquet("ssh_attacks_decoded.parquet")

In [ ]:
df_decoded_splitted = df_decoded.copy()
df_decoded_splitted["full_session"] = df_decoded_splitted["full_session"].apply(lambda session: split_by_vocabulary(session))
df_decoded_splitted.to_parquet("ssh_attacks_decoded_splitted.parquet")